In [4]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, regexp_replace
spark = SparkSession.builder.master('local').appName('test').getOrCreate()
df = spark.read.option('header', True).option('inferSchema', True).csv('./zhaopin.txt') \
    .withColumn('min_salary1', split(col('salary'), '-')[0]) \
    .withColumn('max_salary1', split(col('salary'), '-')[1]) \
    .withColumn('min_salary', regexp_replace(col('min_salary1'), "k", "")) \
    .withColumn('max_salary', regexp_replace(col('max_salary1'), "k", "")) \
    .select('positionName','salary','min_salary','max_salary','workYear','city','companyShortName','companySize','district','financeStage','industryField','thirdType','resumeProcessDay','resumeProcessRate')
df.show()


+--------------------------+-------+----------+----------+--------+----+--------------------------+-----------+--------+------------+---------------------+---------+----------------+-----------------+
|              positionName| salary|min_salary|max_salary|workYear|city|          companyShortName|companySize|district|financeStage|        industryField|thirdType|resumeProcessDay|resumeProcessRate|
+--------------------------+-------+----------+----------+--------+----+--------------------------+-----------+--------+------------+---------------------+---------+----------------+-----------------+
|       Senior Data Analyst|15k-30k|        15|        30|   3-5年|北京|                  AppAnnie|  150-500人|  东城区|   D轮及以上|           移动互联网| 数据分析|               1|              100|
|                数据分析师|15k-20k|        15|        20|   3-5年|北京|北京合生活网络科技有限公司|  150-500人|  朝阳区|  不需要融资|             消费生活| 数据分析|               2|               61|
|                数据分析师|14k-25k|        14|        25|

In [6]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, regexp_replace
from pyspark.sql.types import IntegerType
spark = SparkSession.builder.master('local').appName('test').getOrCreate()
df = spark.read.option('header', True).option('inferSchema', True).csv('./zhaopin.txt') \
    .withColumn('min_salary1', split(col('salary'), '-')[0]) \
    .withColumn('max_salary1', split(col('salary'), '-')[1]) \
    .withColumn('min_salary', regexp_replace(col('min_salary1'), "k", "").cast(IntegerType())) \
    .withColumn('max_salary', regexp_replace(col('max_salary1'), "k", "").cast(IntegerType())) \
    .select('positionName','salary','min_salary','max_salary','workYear','city','companyShortName','companySize','district','financeStage','industryField','thirdType','resumeProcessDay','resumeProcessRate')

min_avg = df.groupBy('city') \
    .sum('min_salary') \
    .withColumnRenamed('sum(min_salary)','min_salary')

max_avg = df.groupBy('city') \
    .sum('max_salary') \
    .withColumnRenamed('sum(max_salary)','max_salary')

min_avg.join(max_avg, ['city'], 'left').show()


+----+----------+----------+
|city|min_salary|max_salary|
+----+----------+----------+
|杭州|      2190|      3658|
|成都|       379|       649|
|深圳|      4107|      7163|
|重庆|        60|       110|
|上海|      6201|     10759|
|北京|     11214|     19560|
|广州|      1919|      3284|
|武汉|       447|       779|
+----+----------+----------+



In [8]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('test').getOrCreate()
df = spark.read.option('header', True).option('inferSchema', True).csv('./zhaopin.txt')

df.groupBy('city', 'workYear').count().show(n=100)


+----+----------+-----+
|city|  workYear|count|
+----+----------+-----+
|成都|      不限|   10|
|武汉|应届毕业生|    3|
|成都|应届毕业生|    1|
|广州|    5-10年|   18|
|北京|      不限|   67|
|杭州|    5-10年|   26|
|北京|     1-3年|  168|
|成都|    5-10年|    9|
|深圳|      不限|   26|
|北京|  10年以上|    4|
|上海|      不限|   42|
|重庆|     3-5年|    3|
|深圳|   1年以下|    4|
|武汉|      不限|    6|
|杭州|      不限|   11|
|广州|      不限|   26|
|杭州|应届毕业生|    5|
|北京|   1年以下|    8|
|成都|   1年以下|    2|
|深圳|    5-10年|   34|
|北京|    5-10年|  113|
|深圳|  10年以上|    1|
|杭州|   1年以下|    2|
|杭州|     3-5年|   72|
|上海|应届毕业生|   29|
|广州|     3-5年|   57|
|北京|     3-5年|  287|
|重庆|      不限|    2|
|广州|   1年以下|    5|
|杭州|     1-3年|   30|
|广州|     1-3年|   52|
|成都|     1-3年|   13|
|上海|     1-3年|  132|
|武汉|    5-10年|   11|
|武汉|     1-3年|   10|
|上海|    5-10年|   57|
|武汉|     3-5年|   14|
|深圳|     3-5年|  129|
|深圳|     1-3年|   75|
|北京|应届毕业生|   63|
|上海|     3-5年|  164|
|深圳|应届毕业生|    6|
|上海|   1年以下|    9|
|重庆|     1-3年|    3|
|广州|应届毕业生|   12|
|成都|     3-5年|    8|
+----+--------